<a href="https://colab.research.google.com/github/vaasu14/pyspark/blob/main/Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2//spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz

In [2]:
cd /content/spark-3.1.2-bin-hadoop3.2

/content/spark-3.1.2-bin-hadoop3.2


In [3]:

!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark').getOrCreate()


In [4]:
cd /content/spark-3.1.2-bin-hadoop3.2

/content/spark-3.1.2-bin-hadoop3.2


In [5]:
from google.colab import files
upload=files.upload()

Saving customer_segmentation.csv to customer_segmentation (2).csv


In [6]:
data=spark.read.csv('/content/spark-3.1.2-bin-hadoop3.2/customer_segmentation.csv',inferSchema=True,header=True)

In [7]:
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Customer Id: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Edu: integer (nullable = true)
 |-- Years Employed: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Card Debt: double (nullable = true)
 |-- Other Debt: double (nullable = true)
 |-- Defaulted: double (nullable = true)
 |-- DebtIncomeRatio: double (nullable = true)



In [8]:
data.columns

['_c0',
 'Customer Id',
 'Age',
 'Edu',
 'Years Employed',
 'Income',
 'Card Debt',
 'Other Debt',
 'Defaulted',
 'DebtIncomeRatio']

In [9]:
final_data=['Age','Edu','Years Employed','Income','Card Debt','Other Debt','Defaulted','DebtIncomeRatio']

In [10]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+-----------------+
|summary|               _c0|       Customer Id|               Age|               Edu|   Years Employed|            Income|         Card Debt|        Other Debt|          Defaulted|  DebtIncomeRatio|
+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+-----------------+
|  count|               850|               850|               850|               850|              850|               850|               850|               850|                700|              850|
|   mean|             424.5|             425.5|35.029411764705884|1.7105882352941177|8.565882352941177|46.675294117647056| 1.576819999999999| 3.078772941176469|0.26142857142857145|10.17164705882353|
| std

In [11]:
data=data.na.drop()

In [12]:
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+---+-----------+---+---+--------------+------+---------+----------+---------+---------------+
|_c0|Customer Id|Age|Edu|Years Employed|Income|Card Debt|Other Debt|Defaulted|DebtIncomeRatio|
+---+-----------+---+---+--------------+------+---------+----------+---------+---------------+
|  0|          0|  0|  0|             0|     0|        0|         0|        0|              0|
+---+-----------+---+---+--------------+------+---------+----------+---------+---------------+



In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
assembler=VectorAssembler(inputCols=final_data,outputCol='features')

In [15]:
output=assembler.transform(data)

In [16]:
output.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Customer Id: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Edu: integer (nullable = true)
 |-- Years Employed: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Card Debt: double (nullable = true)
 |-- Other Debt: double (nullable = true)
 |-- Defaulted: double (nullable = true)
 |-- DebtIncomeRatio: double (nullable = true)
 |-- features: vector (nullable = true)



In [17]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler

In [36]:
scaler=StandardScaler(inputCol='features',outputCol='scaledfeatures')

In [37]:
scaler_model=scaler.fit(output)

In [39]:
final=scaler_model.transform(output)

In [40]:
final.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Customer Id: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Edu: integer (nullable = true)
 |-- Years Employed: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Card Debt: double (nullable = true)
 |-- Other Debt: double (nullable = true)
 |-- Defaulted: double (nullable = true)
 |-- DebtIncomeRatio: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledfeatures: vector (nullable = true)



In [41]:
kmean3 = KMeans(featuresCol='scaledfeatures',k=3)

In [42]:
kmean4= KMeans(featuresCol='scaledfeatures',k=4)

In [43]:
kmean2= KMeans(featuresCol='scaledfeatures',k=2)

In [45]:
kmodel_K2 = kmean2.fit(final)

In [47]:
kmodel_K3 = kmean3.fit(final)

In [46]:
kmodel_K4= kmean4.fit(final)

In [49]:
kmodel_K3.transform(final).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  108|
|         2|  430|
|         0|  162|
+----------+-----+



In [51]:
kmodel_K4.transform(final).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  334|
|         3|   32|
|         2|  168|
|         0|  166|
+----------+-----+



In [52]:
kmodel_K2.transform(final).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  196|
|         0|  504|
+----------+-----+

